# Covid-19-Guinea

Creating dashboards to follow the evolution of covid-19 in Guinea. These dashboards will idealy be hosted to Github Pages and integrated to a wordpress website named : www.moucoronakhaninma.com. 

## Imports

### Sys and Python

In [59]:
import sys
import os

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import datetime
import git
import glob
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Graphics

In [155]:
%matplotlib inline

import cufflinks as cf
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import plotly as py
import plotly.express as px
import plotly.io as pio
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly.graph_objs import graph_objs

### Source files

In [4]:
%load_ext autoreload
%autoreload 2

- Global variables

In [263]:
COVID_19_DATA_PATH = '../../COVID-19/'
DAILY_REPORTS_PATH = COVID_19_DATA_PATH+'csse_covid_19_data/csse_covid_19_daily_reports/'
GUINEA_COVID_19_START_DATE = '03-13-2020'
ANSS_URL = 'https://anss-guinee.org'
GUINEA_COVID_19_DATA_PATH = '../data/guinea.csv'

## Scraping ANSS

This way, we will run a script to automatically get the updated data of covid-19 evolution each day from the ANSS website.

In [209]:
source_code = requests.get(ANSS_URL)
plain_text = source_code.text
soup = BeautifulSoup(plain_text, 'html.parser')
data = soup.findAll('span', attrs={'class':'count-text'})
confirmed = int(data[0].attrs['data-stop'])
death = int(data[1].attrs['data-stop'])
healed = int(data[2].attrs['data-stop'])
new_line = [confirmed, death, healed]

## Getting historic data from Covid-19 github repo

In [5]:
repo = git.Repo(COVID_19_DATA_PATH)
repo.remotes.origin.pull()

In [29]:
files = glob.glob(DAILY_REPORTS_PATH+'*.csv')
files.sort(key=os.path.getmtime)
index_start = files.index(DAILY_REPORTS_PATH+GUINEA_COVID_19_START_DATE+'.csv')
files = files[index_start:]

In [56]:
pd.read_csv(files[0]).columns

Index(['Province/State', 'Country/Region', 'Last Update', 'Confirmed',
       'Deaths', 'Recovered', 'Latitude', 'Longitude'],
      dtype='object')

In [57]:
pd.read_csv(files[-1]).columns

Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key'],
      dtype='object')

In [157]:
columns = ['Date', 'Cas confirmés', 'Décès', 'Guéris']
df_guinea = pd.DataFrame(columns=columns)

In [158]:
for f in files:
    df = pd.read_csv(f)
    date = f.split('/')[-1].split('.')[0]
    date = datetime.datetime.strptime(date, '%m-%d-%Y').strftime("%d-%m-%Y")
    try:
        cases = df[df['Country_Region'] == 'Guinea'].values[0][7:10]
    except:
        cases = df[df['Country/Region'] == 'Guinea'].values[0][3:6]
    
    line = list(cases)
    #datetime.datetime.strptime(date, '%d-%m-%Y').date()
    line.insert(0, date)
    df_guinea = df_guinea.append(pd.DataFrame(line, index=columns).T)
    df_guinea.index = range(len(df_guinea))

In [159]:
df_guinea['Cas hospitalisés'] = df_guinea['Cas confirmés'] - df_guinea['Décès'] - df_guinea['Guéris']

In [160]:
df_guinea['Nouveaux cas'] = 0
df_guinea['Nouveaux décès'] = 0
df_guinea['Nouveaux guéris'] = 0
df_guinea.iloc[0, 5] = df_guinea.iloc[0, 1]
df_guinea.iloc[1:, 5] = df_guinea.iloc[1:, 1] - df_guinea.iloc[0:-1, 1].values
df_guinea.iloc[1:, 6] = df_guinea.iloc[1:, 2] - df_guinea.iloc[0:-1, 2].values
df_guinea.iloc[1:, 7] = df_guinea.iloc[1:, 3] - df_guinea.iloc[0:-1, 3].values

In [192]:
df_guinea

,Date,Cas confirmés,Décès,Guéris,Cas hospitalisés,Nouveaux cas,Nouveaux décès,Nouveaux guéris
0,13-03-2020,1,0,0,1,1,0,0
1,14-03-2020,1,0,0,1,0,0,0
2,15-03-2020,1,0,0,1,0,0,0
3,16-03-2020,1,0,0,1,0,0,0
4,17-03-2020,1,0,0,1,0,0,0
5,18-03-2020,1,0,0,1,0,0,0
6,19-03-2020,1,0,0,1,0,0,0
7,20-03-2020,1,0,0,1,0,0,0
8,21-03-2020,2,0,0,2,1,0,0
9,22-03-2020,2,0,0,2,0,0,0


In [210]:
ch = new_line[0] - new_line[1] - new_line[2]
nc = new_line[0] - df_guinea.iloc[-1, 1]
nd = new_line[1] - df_guinea.iloc[-1, 2]
ng = new_line[2] - df_guinea.iloc[-1, 3]
new_line.extend([ch, nc, nd, ng])
new_line.insert(0, datetime.datetime.today().strftime('%d-%m-%Y'))

In [211]:
new_line

['17-04-2020', 477, 3, 59, 415, 39, 2, 10]

In [212]:
df_guinea = df_guinea.append(pd.DataFrame(new_line, index=df_guinea.columns).T)

In [258]:
x = df_guinea['Date'].values

fig = go.Figure(data=[
    go.Bar(name='Décès', x=x, y=df_guinea['Décès'].values, marker_color='red'),
    go.Bar(name='Guéris', x=x, y=df_guinea['Guéris'].values, marker_color='darkgreen'),
    go.Bar(name='Cas hospitalisés', x=x, y=df_guinea['Cas hospitalisés'], marker_color='gold')
])
fig.add_trace({'x':df_guinea['Date'], 'y':df_guinea['Cas confirmés'], 'name':'Cas confirmés'})

# Change the bar mode
fig.update_layout(template='plotly_dark', 
                  barmode='stack', hovermode='x', 
                  xaxis_tickangle=-60,
                  legend_orientation="h",
                  legend=dict(x=0, y=-0.3),
                  margin=dict(t=40, b=0, l=25, r=3),
                  title='Évolution du Covid-19 en Guinée : données cumulées')
fig.update_yaxes(automargin=True)
fig.show()

In [264]:
df_guinea = pd.read_csv(GUINEA_COVID_19_DATA_PATH)

In [266]:
df_guinea.tail()

,Date,Cas confirmés,Décès,Guéris,Cas hospitalisés,Nouveaux cas,Nouveaux décès,Nouveaux guéris
32,14-04-2020,363,0,31,332,44,0,14
33,15-04-2020,404,1,31,372,41,1,0
34,16-04-2020,438,1,49,388,34,0,18
35,17-04-2020,477,3,59,415,39,2,10
36,18-04-2020,518,3,65,450,41,0,6


In [385]:
x = df_guinea['Date'].values

fig = go.Figure()

config = {'displayModeBar': True}

fig.add_traces(data=[
    go.Bar(name='Décès', x=x, y=df_guinea['Décès'].values, marker_color='red'),
    go.Bar(name='Guéris', x=x, y=df_guinea['Guéris'].values, marker_color='darkgreen'),
    go.Bar(name='Cas hospitalisés', x=x, y=df_guinea['Cas hospitalisés'], marker_color='gold')
])

fig.add_trace({'x':df_guinea['Date'], 'y':df_guinea['Cas confirmés'], 'name':'Cas confirmés'})

fig.add_traces(data=[
    go.Bar(name='Nouveaux décès', x=x, y=df_guinea['Nouveaux décès'].values, marker_color='red',
          visible=False),
    go.Bar(name='Nouveaux guéris', x=x, y=df_guinea['Nouveaux guéris'].values, marker_color='darkgreen',
          visible=False),
    go.Bar(name='Nouveaux cas', x=x, y=df_guinea['Nouveaux cas'], marker_color='gold',
          visible=False)
])


fig.update_yaxes(automargin=True)

# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            active=0,
            buttons=list([
                dict(
                    label="Cumul",
                    method="update",
                    args=[{"visible": [True, True, True, True, False, False, False]}],        
                ),
                dict(
                    label="Par jour",
                    method="update",
                    args=[{"visible": [False, False, False, False, True, True, True]},
                           {"title": "Évolution du Covid-19 en Guinée : nouvelles stats par jour",
                            }],                    
                )
            ]),
            direction="down",
            showactive=True,
            x=0.01,
            xanchor="center",
            y=1.02,
            yanchor='middle',
            bgcolor='darkred',
            font=dict(color='gray', size=14)
        ),
    ],
    template='plotly_dark',
    barmode='stack', hovermode='x', 
    xaxis_tickangle=-60,
    legend_orientation="h",
    legend=dict(x=0, y=-0.3),
    margin=dict(t=40, b=0, l=25, r=3),
    title=dict(
        text='Évolution du Covid-19 en Guinée : données cumulées', x=0.5, y=0.1,
        font=dict(size=13)),
)



In [386]:
pio.write_html(fig, file='../index.html', auto_open=True)

tls.get_embed('https://atidiane.github.io/covid-19-guinea/')

In [255]:
with open('../Test.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

11556

11556